In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

from easydict import EasyDict

import scipy.stats as ss

In [233]:
from glob import glob

In [196]:
from multiinstance.data.realData import buildDataset as buildReal
from multiinstance.utils import addTransformScores, addGlobalEsts, getBagAlphaHats,getTransformScores

In [180]:
mu1 = 2
mu0 = 1

In [181]:
NP= 2
NU = 10
alpha=.25
S0 = NP / (NP + NU)
S1 = 1 - S0
C = np.random.normal(loc=5,scale=1, size=NP)
M,mL = getM(NU,alpha)

In [182]:
def getM(NU,alpha):
    M = np.concatenate((np.random.normal(loc=mu1,size=int(alpha*NU)),
                        np.random.normal(loc=mu0, size=int((1-alpha) * NU))))
    mL = np.concatenate((np.ones(int(alpha*NU)),
                                   np.zeros(int((1-alpha) * NU))))
    return M, mL

In [189]:
def f(x,alpha_):
    return alpha_ * ss.norm.pdf(x,loc=mu1,scale=1) + (1 - alpha_) * ss.norm.pdf(x,loc=mu0)

def g(x):
    return ss.norm.pdf(x,loc=mu1)

def tau(x,alpha_):
    return S1 / ((f(x,alpha_) / g(x)) * S0 + S1)

def posterior(x,alpha_,tau_=None, sos1=None):
    if tau_ is None:
        tau_ = tau(x,alpha_)
    if sos1 is None:
        sos1 = S0/S1
    return alpha_ * sos1 * (tau_ / (1 - tau_))

def truePosterior(x,alpha_):
    f0 = ss.norm.pdf(x,loc=mu0)
    f1 = ss.norm.pdf(x,loc=mu1)
    return (f1 * alpha_) / (alpha_ * f1 + (1 - alpha_) * f0)

In [190]:
np.allclose(posterior(C,alpha), truePosterior(C,alpha)), np.allclose(posterior(M,alpha), truePosterior(M,alpha))

(True, True)

In [197]:
def auc(scores, alphas, ds):
    posts = [posterior(xi,alphai) for xi, alphai in zip(scores, alphas)]
    posteriors = np.concatenate(posts)
    labels = np.concatenate([ds.hiddenLabels[i][:ds.numU[i]] for i in range(ds.N)])
    return roc_auc_score(labels, posteriors)

In [192]:
dsi = EasyDict()
dsi.hiddenLabels = [np.concatenate((np.ones(int(alpha*NU)),
                                   np.zeros(int((1-alpha) * NU))))]
dsi.numU = [dsi.hiddenLabels[i].shape[0] for i in range(len(dsi.hiddenLabels))]
dsi.N = 1

auc([M], [alpha], dsi)

0.7142857142857143

In [240]:
d2 = EasyDict()
d2.N = 100
alphas = np.random.beta(2,2,size=d2.N)
d2.numU = 1 + np.random.poisson(lam=100,size=d2.N)
scores, d2.hiddenLabels = list(zip(*[getM(n,a) for n,a in zip(d2.numU, alphas)]))
print("local: {:.3f}".format(auc(scores, alphas+ np.random.normal(0,.1,size=d2.N), d2)))
gAlpha = np.dot(alphas, d2.numU) / np.sum(d2.numU) + np.random.normal(0,.06)
print("global: {:.3f}".format(auc(scores, np.ones(d2.N)*gAlpha, d2)))

TypeError: 'str' object is not callable

In [235]:
def aucReal(ds, alphas_):
    _,taus = list(zip(*[getTransformScores(ds,i) for i in range(ds.N)]))
    s0s1 = ds.numU / ds.numP
#     pvals = [a * s * (t / (1 - t)) for a,s,t in zip(alphas_, s0s1, taus)]
    pvals = []
    for a,s,t in zip(alphas_,s0s1, taus):
        p = a * s * (t / (1-t))
        p[np.isinf(p)] = 1
        pvals.append(p)
    posteriors = np.concatenate(pvals)
    labels = np.concatenate([ds.hiddenLabels[i][:ds.numU[i]] for i in range(ds.N)])
    return roc_auc_score(labels,posteriors)

In [243]:
a = {"local":[],
     "global":[]}
     
for f in glob("/ssdata/ClassPriorEstimationPrivate/data/rawDatasets/*.mat"):
    dsi = buildReal(f,16,
                    alphaDistr=lambda: np.random.uniform(.25,.75),
                    nPDistr=lambda: 1 + np.random.poisson(25),
                    nUDistr=lambda: 1 + np.random.poisson(50))
    dsi = addTransformScores(dsi)
    dsi = addGlobalEsts(dsi,reps=100)
    dsi.alphaHats, dsi.curves = getBagAlphaHats(dsi,
                                                numbootstraps=100)
    a["local"].append(aucReal(dsi, dsi.alphaHats.mean(1)))
    a["global"].append(aucReal(dsi, dsi.globalAlphaHats.mean() * np.ones(dsi.N)))
    print("local: {:.3f}    global: {:.3f}".format(a["local"][-1],
                                                   a["global"][-1]))
    print("~~~~ AVG: local: {:.3f}    global: {:.3f}".format(np.mean(a["local"]),
                                                   np.mean(a["global"])))

local: 0.916    global: 0.938
~~~~ AVG: local: 0.916    global: 0.938


local: 0.687    global: 0.672
~~~~ AVG: local: 0.802    global: 0.805


local: 0.958    global: 0.954
~~~~ AVG: local: 0.854    global: 0.855


local: 0.971    global: 0.969
~~~~ AVG: local: 0.883    global: 0.883


local: 0.818    global: 0.845
~~~~ AVG: local: 0.870    global: 0.876


...

local: 0.909    global: 0.918
~~~~ AVG: local: 0.884    global: 0.889


local: 0.838    global: 0.871
~~~~ AVG: local: 0.882    global: 0.889


local: 0.954    global: 0.955
~~~~ AVG: local: 0.885    global: 0.891


/home/dz/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/dz/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys


local: 0.650    global: 0.631
~~~~ AVG: local: 0.877    global: 0.882


In [241]:
a = {"local":[],
     "global":[]}
     
for f in glob("/ssdata/ClassPriorEstimationPrivate/data/rawDatasets/*.mat"):
    dsi = buildReal(f,16,
                    alphaDistr=lambda: np.random.uniform(.25,.75),
                    nPDistr=lambda: 1 + np.random.poisson(25),
                    nUDistr=lambda: 1 + np.random.poisson(50))
    dsi = addTransformScores(dsi)
    #dsi = addGlobalEsts(dsi,reps=100)
    #dsi.alphaHats, dsi.curves = getBagAlphaHats(dsi,
                                                #numbootstraps=100)
    globalAlpha = np.dot(dsi.trueAlphas.flatten(), dsi.numU) / np.sum(dsi.numU)
    
    a["local"].append(aucReal(dsi, dsi.trueAlphas.flatten()))
    a["global"].append(aucReal(dsi, globalAlpha * np.ones(dsi.N)))
    print("local: {:.3f}    global: {:.3f}".format(a["local"][-1],
                                                   a["global"][-1]))
    print("~~~~ AVG: local: {:.3f}    global: {:.3f}".format(np.mean(a["local"]),
                                                   np.mean(a["global"])))

local: 0.926    global: 0.948
~~~~ AVG: local: 0.926    global: 0.948


local: 0.729    global: 0.710
~~~~ AVG: local: 0.827    global: 0.829


local: 0.971    global: 0.968
~~~~ AVG: local: 0.875    global: 0.876


local: 0.978    global: 0.975
~~~~ AVG: local: 0.901    global: 0.900


local: 0.821    global: 0.771
~~~~ AVG: local: 0.885    global: 0.875


...

local: 0.881    global: 0.859
~~~~ AVG: local: 0.901    global: 0.892


local: 0.838    global: 0.798
~~~~ AVG: local: 0.899    global: 0.888


local: 0.966    global: 0.962
~~~~ AVG: local: 0.901    global: 0.891


/home/dz/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/dz/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys


local: 0.668    global: 0.500
~~~~ AVG: local: 0.893    global: 0.878


In [248]:
np.abs(dsi.alphaHats.mean(1) - dsi.trueAlphas.flatten()).max()

0.2466858253762934

In [249]:
np.abs(globalAlpha - dsi.trueAlphas.flatten()).max()

0.2075509776979792